In [1]:
from numpy import *
from scipy.sparse import *

In [2]:
def MAP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

SKIP FROM HERE 

In [3]:
allplt= load_npz("all_playlists_with_tags.npz")

In [4]:
all_playlists_with_tracks=load_npz("all_playlists_with_tracks.npz")

In [5]:
target_pl = genfromtxt("target_playlists.csv",skip_header=1)

In [10]:
non_target_indexes = [all_playlists_with_tracks.getrow(0).data[0]]
non_target_pl_tags = allplt.getrow(0)
for i in range(1, 45649):
    if(i%1000==0):
        print(str(i))
    if(not isin(all_playlists_with_tracks.getrow(i).data[0], target_pl)):
        non_target_indexes.append(all_playlists_with_tracks.getrow(i).data[0])
        non_target_pl_tags = vstack([non_target_pl_tags, allplt.getrow(i)])
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


In [11]:
non_target_pl_tags

<35649x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 1226848 stored elements in Compressed Sparse Row format>

In [17]:
shape(non_target_indexes)

(35649,)

In [15]:
save("non_target_pl_indexes.npy", non_target_indexes)

In [16]:
save_npz("non_target_pl_with_tags.npz", non_target_pl_tags)

TO HERE SKIP

In [10]:
non_target_indexes = load("non_target_pl_indexes.npy")

In [11]:
non_target_pl_tags= load_npz("non_target_pl_with_tags.npz")

In [12]:
non_target_pl_tags = non_target_pl_tags.tocsc()

In [15]:
type(non_target_pl_tags)

scipy.sparse.csc.csc_matrix

In [16]:
non_target_pl_tags.shape

(35649, 276615)

In [14]:
tracks_with_tags=load_npz("tracks_with_tags_bincount_ordered.npz")
tracks_with_tags.shape

(100000, 276615)

In [22]:
for i in range(1,276615):#la prima colonna conta i tag con valore zero, la saltiamo
    if(tracks_with_tags.getcol(i).size!=0): #troviamo la prima colonna con tag nonzero
        print(i)
        break

22


In [28]:
non_target_pl_tags_reduced = non_target_pl_tags.getcol(22)#partiamo dalla prima colonna con tag nonzero
for i in range(23, 276615):
    if(i%100000==0):
        print(str(i))
    if(tracks_with_tags.getcol(i).size!=0):#il controllo va fatto sulle tracks
        non_target_pl_tags_reduced = hstack([non_target_pl_tags_reduced, non_target_pl_tags.getcol(i)])
    


100000
200000


In [29]:
non_target_pl_tags_reduced

<35649x31900 sparse matrix of type '<class 'numpy.float64'>'
	with 1217569 stored elements in Compressed Sparse Column format>

In [32]:
save_npz("non_target_pl_tags_reduced.npz",non_target_pl_tags_reduced)